In [ ]:
# 训练顺序：
# 1、train_nq_binencoder.sh 训练完后，encode 文章，预测训练集，用于生成hard negative
# 2、train_nq_rereaker.sh 训练个reranker，用于知识蒸馏。
# 3、nq_gen_kd_teacher_scores.sh 得到老师模型的分数，用于后续步骤。
# 4、train_nq_kd.sh 训练知识蒸馏后的结果

In [1]:
# 处理数据文件pasage_w100
import json
with open('/home/shishijie/workspace/project/DPR/downloads/wiki/downloads/data/wikipedia_split/psgs_w100.tsv','r') as fr:
    with open('/home/shishijie/workspace/project/unilm/simlm/data/dpr/passages.jsonl','w') as fw:
        for line in fr.readlines():
            line=line.strip().split('\t')
            fw.write(json.dumps({
                'id': line[0],
                'contents': line[1],
                'title': '-'
            })+'\n')


In [2]:
# nq_train nq_dev 转为jsonl
def write_jsonl(filename):
    with open(f'./data/dpr/{filename}.json', 'r') as fr:
        data=json.loads(fr.read())

    with open(f'./data/dpr/{filename}.jsonl', 'w') as fw:
        for item in data:
            
            fw.write(json.dumps(item) + '\n')

write_jsonl('nq-train')
# write_jsonl('nq-dev')
# write_jsonl('nq-adv-hn-train')

In [2]:
import json
def trans(filename, type):
    if type == 'json2jsonl':
        
        # query2id
        data_dic={}
        with open(f'/home/shishijie/workspace/project/DPR/downloads/wiki/downloads/data/retriever/qas/{filename}.csv', 'r') as fr:
            for idx, line in enumerate(fr.readlines()):
                line = line.strip().split('\t')
                data_dic[line[0]] = idx
        
        # 读取带有正负样本的训练集
        with open(f'./data/dpr/{filename}.json', 'r') as fr:
            data_json=json.loads(fr.read())
            
        with open(f'./data/dpr/{filename}.jsonl', 'w') as fw:
            for item in data_json:
                query = item['question']
                query_id = data_dic[query]
                positives = {'doc_id': [], 'score': []}
                positive_ctxs = item['positive_ctxs']
                for positive_ctx in positive_ctxs:
                    positives['doc_id'].append(positive_ctx['passage_id'])
                    positives['score'].append(positive_ctx['score'])
                
                negatives = {'doc_id': [], 'score': []}
                negative_ctxs = item['negative_ctxs']
                for negative_ctx in negative_ctxs:
                    negatives['doc_id'].append(negative_ctx['passage_id'])
                    negatives['score'].append(negative_ctx['score'])
                
                fw.write(json.dumps({
                    'query_id': query_id,
                    'query': query,
                    'positives': positives,
                    'negatives': negatives
                }) + '\n')
                
    elif type == 'query2tsv':  
        with open(f'/home/shishijie/workspace/project/DPR/downloads/wiki/downloads/data/retriever/qas/{filename}.csv', 'r') as fr:
            filename = filename.replace('-', '_')  
            with open(f'./data/dpr/{filename}_queries.tsv', 'w') as fw:
                for line in fr.readlines():
                    fw.write(line)
    
    # 挖掘困难负样本       
    elif type == 'minedhn':
        data_dic={}
        with open(f'/home/shishijie/workspace/project/DPR/downloads/wiki/downloads/data/retriever/qas/{filename}.csv', 'r') as fr:
            for idx, line in enumerate(fr.readlines()):
                line = line.strip().split('\t')
                data_dic[line[0]] = str(idx)
        
        with open(f'./data/dpr/{filename}.json', 'r') as fr:
            data_json=json.loads(fr.read())
        
        with open(f'/home/shishijie/workspace/project/unilm/simlm/checkpoint/nq_rerank_2023-12-18-2154.01/checkpoint-20000/nq_train.dpr.json', 'r') as fr:
            data_predict = json.loads(fr.read())
        
        with open(f'./data/dpr/{filename}_distill.jsonl', 'w') as fw:
            for item in data_json:
                query = item['question']
                query_id = data_dic[query]
                positives = {'doc_id': [], 'score': []}
                positive_ctxs = item['positive_ctxs']
                for positive_ctx in positive_ctxs:
                    positives['doc_id'].append(positive_ctx['passage_id'])
                    positives['score'].append(positive_ctx['score'])
                    
                negatives = {'doc_id': [], 'score': []}
                predict = data_predict[query]
                hn_count = 15
                for tmp_dic in predict['contexts']:
                    if not tmp_dic['has_answer']:
                        negatives['doc_id'].append(str(tmp_dic['docid']))
                        negatives['score'].append(float(tmp_dic['score']) if tmp_dic['score'] else 0)
                        hn_count -= 1
                    if not hn_count:
                        break
                    
                negative_ctxs = item['negative_ctxs']
                for negative_ctx in negative_ctxs:
                    negatives['doc_id'].append(negative_ctx['passage_id'])
                    negatives['score'].append(negative_ctx['score'])
                
                   
                fw.write(json.dumps({
                    'query_id': query_id,
                    'query': query,
                    'positives': positives,
                    'negatives': negatives
                }) + '\n')
                
                
trans('nq-train', 'minedhn')